# Azure Cognitive Service Computer Vision SDK for Python

https://docs.microsoft.com/en-us/azure/cognitive-services/Computer-vision/quickstarts-sdk/python-sdk

### SDK installation

`pip` command is used to install Python packages. Note that '!' is used to execute a system command.

In [1]:
# Uncomment '#' and execute the following command if you have not executed previously

#!pip install azure-cognitiveservices-vision-computervision

    100% |████████████████████████████████| 92kB 3.9MB/s ta 0:00:01


### Getting credentials for key and endpoint

Note that `Notebook` environment does not support `environment variables` well - recommend to copy & paste endpoint & key from your Azure Portal.

### Creating client

In [5]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

# Get endpoint and key from environment variables
endpoint = 'https://westus.api.cognitive.microsoft.com/'
key = '[your subscription key]' # Replace with your subscription key

# Set credentials
credentials = CognitiveServicesCredentials(key)

# Create client
client = ComputerVisionClient(endpoint, credentials)

### Example 1: Analyze an image

In [6]:
url = "https://azurecomcdn.azureedge.net/cvt-dd056dd30ec0bcbe16e12033e0518062de64be7165b3911783d38c31675b27ba/images/shared/cognitive-services-demos/analyze-image/analyze-6.jpg"

language = "en"
max_descriptions = 3

try:
    image_analysis_image_type = client.analyze_image(url,visual_features=[VisualFeatureTypes.image_type])
    image_analysis_category = client.analyze_image(url,visual_features=[VisualFeatureTypes.categories])
    image_analysis_adult = client.analyze_image(url,visual_features=[VisualFeatureTypes.adult])
    image_analysis_face = client.analyze_image(url,visual_features=[VisualFeatureTypes.faces])

    print(image_analysis_image_type.image_type.as_dict())
    for category in image_analysis_category.categories:
        print(category.as_dict())
    print(image_analysis_adult.adult.as_dict())
    for face in image_analysis_face.faces:
        print(face.as_dict())

except HTTPFailure as e:
    if e.status_code == 401:
        print("Error unauthorized. Make sure your key and endpoint are correct.")
    else:
        raise

{'clip_art_type': 0, 'line_drawing_type': 0}
{'name': 'people_group', 'score': 0.97265625}
{'is_adult_content': False, 'is_racy_content': False, 'adult_score': 0.0013711383799090981, 'racy_score': 0.0046537225134670734}
{'age': 43, 'gender': 'Male', 'face_rectangle': {'left': 118, 'top': 159, 'width': 95, 'height': 95}}
{'age': 54, 'gender': 'Male', 'face_rectangle': {'left': 490, 'top': 111, 'width': 91, 'height': 91}}
{'age': 56, 'gender': 'Female', 'face_rectangle': {'left': 19, 'top': 155, 'width': 82, 'height': 82}}
{'age': 33, 'gender': 'Female', 'face_rectangle': {'left': 386, 'top': 166, 'width': 81, 'height': 81}}
{'age': 17, 'gender': 'Female', 'face_rectangle': {'left': 235, 'top': 159, 'width': 76, 'height': 76}}
{'age': 7, 'gender': 'Female', 'face_rectangle': {'left': 323, 'top': 163, 'width': 68, 'height': 68}}


### Example 2: Get text from image

In [7]:
# import models
from azure.cognitiveservices.vision.computervision.models import TextRecognitionMode
from azure.cognitiveservices.vision.computervision.models import TextOperationStatusCodes
import time

url = "https://azurecomcdn.azureedge.net/cvt-dd056dd30ec0bcbe16e12033e0518062de64be7165b3911783d38c31675b27ba/images/shared/cognitive-services-demos/read-text/read-2.jpg"
mode = TextRecognitionMode.printed
raw = True
custom_headers = None
numberOfCharsInOperationId = 36

# Async SDK call
rawHttpResponse = client.batch_read_file(url, mode, custom_headers,  raw)

# Get ID from returned headers
operationLocation = rawHttpResponse.headers["Operation-Location"]
idLocation = len(operationLocation) - numberOfCharsInOperationId
operationId = operationLocation[idLocation:]

# SDK call
while True:
    result = client.get_read_operation_result(operationId)
    if result.status not in ['NotStarted', 'Running']:
        break
    time.sleep(1)

# Get data
if result.status == TextOperationStatusCodes.succeeded:
    for textResult in result.recognition_results:
        for line in textResult.lines:
            print(line.text)
            #print(line.bounding_box)

CLOSED
WHEN ONE DOOR CLOSES, ANOTHER
OPENS.ALL YOU HAVE TO DO IS WALK IN
